In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import copy

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()  
])

transform_val = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor()
])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
])

In [ ]:
train_set = datasets.ImageFolder("/kaggle/input/chest-xray-pneumonia/chest_xray/train", transform= transform_train)
val_set = datasets.ImageFolder("/kaggle/input/chest-xray-pneumonia/chest_xray/val", transform=transform_val)
test_set = datasets.ImageFolder("/kaggle/input/chest-xray-pneumonia/chest_xray/test", transform=transform_test)

Data Visualisation

In [ ]:
num_classes = 2
batch_size = 8

trainloader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(val_set, batch_size = batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_set, batch_size = batch_size, shuffle=False)


In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)

print(images[1].shape)
print(labels[1].item())

print("Classes in training set")
print(train_set.classes)
print("Classes in validation set")
print(val_set.classes)

In [ ]:
def imshow(img, title):
    npimg = img.numpy()
    plt.figure(figsize=(15,7))
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.title(title)
    plt.show()
def show_batch_images(dataloader):
    images, labels = next(iter(dataloader))
    img = torchvision.utils.make_grid(images, padding=25)
    imshow(img, title=["NORMAL" if x==0 else "Pneumonia" for x in labels])
for i in range(4):
    show_batch_images(trainloader)

Transfer Learning and trainiing VGG16

In [ ]:
def evaluation(dataloader, model):
    total, correct =0,0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs, data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return(100 * correct/ total)

In [ ]:
vgg = models.vgg16_bn(pretrained=True)
for param in vgg.parameters():
    param.requires_grad = False


In [ ]:
final_in_features = vgg.classifier[6].in_features
vgg.classifier[6] = nn.Linear(final_in_features, num_classes)
for param in vgg.parameters():
    if param.requires_grad:
        print(param.shape)

In [ ]:
vgg = vgg.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(vgg.parameters(), lr=0.01)

In [ ]:
%%time

loss_epoch_arr = []
max_epochs = 20

min_loss = 10000000

n_iters = np.ceil(5300/batch_size)

for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = vgg(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
        if min_loss > loss.item():
            min_loss = loss.item()
            best_model = copy.deepcopy(vgg.state_dict())
            print('Min loss %0.2f' % min_loss)
        
        if i % 100 == 0:
            print('Iteration: %d/%d, Loss: %0.2f' % (i, n_iters, loss.item()))
            
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
    loss_epoch_arr.append(loss.item())

vgg.load_state_dict(best_model)
print(evaluation(trainloader, vgg), evaluation(valloader, vgg))

In [ ]:
print("Acc on training set is {} and validation set is {}".format(evaluation(trainloader, vgg), evaluation(valloader, vgg)))

In [ ]:
plt.plot(loss_epoch_arr)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("loss vs epochs")
plt.show()